In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

pd.set_option('display.notebook_repr_html', True)
 
%matplotlib inline
plt.style.use('seaborn-white')

C:\Users\apuri4\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Data Cleaning Step

In [242]:
# df = pd.read_csv('drug/final.csv', dtype={ 'Labeler Code' : int, 'Product Code' : int, 'NDC': str})
# df.columns = df.columns.str.replace(' ', '_')
# df.drop(['Latitude', 'Longitude', 'Location'], axis=1, inplace=True)
# unsup = df[df.Suppression_Used == False]
# del(df)
# unsup.drop('Suppression_Used', axis=1, inplace=True)
# # unsup.drop('NDC', axis=1, inplace=True)
# # unsup.drop('Labeler_Code', axis=1, inplace=True)
# # unsup.drop('Product_Code', axis=1, inplace=True)
# unsup['Year'] = unsup.Quarter_Begin_Date.str.extract(pat='(20\d{2})')
# rep_dict = {}
# rep_dict['1/1'] = 1
# rep_dict['4/1'] = 2
# rep_dict['7/1'] = 3
# rep_dict['10/1'] = 4
# unsup['Quarter'] = unsup.Quarter_Begin.apply(lambda x: rep_dict[x])
# unsup.drop(['Quarter_Begin', 'Quarter_Begin_Date'], axis=1, inplace=True)
# # unsup.Package_Size = pd.to_numeric(unsup.Package_Size)


# #State aggregate? 
# xx = unsup[unsup.State == 'XX']
# unsup.drop(unsup[unsup.State == 'XX'].index, inplace=True)

# #drug product labeling api key : vNby8diZCQCNddeSjnBS2SPhXX6hCvFmWv4ZgPvN

# #dealing with Labeler NDC codes
# unsup['Labeler_Code'] = unsup.Labeler_Code.astype('str')
# unsup['Labeler_Code'] = unsup.Labeler_Code.str.rstrip()
# unsup['Labeler_Code'] = unsup.Labeler_Code.str.replace('(^\d{1}$)', '000\\1').str.replace('(^\d{2}$)', '00\\1').str.replace('(^\d{3}$)', '0\\1')
# unsup.drop('NDC', axis=1, inplace=True)

# unsup.Product_Name = unsup.Product_Name.str.rstrip()
# pd.to_pickle(unsup, 'unsuppressed_cleaned_state.pkl')

unsup = pd.read_pickle('unsuppressed_cleaned_state.pkl')


# Calling openFDA API

In [459]:
#Call openFDA API
from urllib.request import Request, urlopen, URLError, HTTPError
from urllib.parse import urlencode
import json
from pprint import pprint
api_key = 'vNby8diZCQCNddeSjnBS2SPhXX6hCvFmWv4ZgPvN'
values = {'brand_name', 'product_type', 
          'generic_name', 'manufacturer_name'}
import time



def get_info(L, P, api_key=api_key, values=values):
    #set endpoint
    NDC = '%s-%s' % (L, P)
    NDC2 = '%s-%s' % (L, P[1:])
    NDC_dict = {}
    endpoint = 'https://api.fda.gov/drug/label.json?search=openfda.product_ndc:"%s"&limit=100' % NDC
    endpoint2 = 'https://api.fda.gov/drug/label.json?search=openfda.product_ndc:"%s"&limit=100' % NDC2
    req = Request(endpoint)
    #
    req.add_header("Authorization", api_key)
    #tries to query the NDC, outs is whatever is loaded from openFDA
    #For each desired value in values, try to query the dictionary for val and put in a new dict
    #If keyerror, aka not in openFDA, put in not found
    
    try: 
        res = urlopen(req)
        outs = json.loads(res.read())
        #Tries to provide returned values, if that column doesnt exist, return null list
        for imp in values:
            try:
                NDC_dict[imp] = outs['results'][0]['openfda'][imp]
            except KeyError:
                NDC_dict[imp] = 'not_found'
                
    #IF query does not register ie. NDC code is wrong, try the shortened P endpoint, if that fails, fuck it            
    except HTTPError as e:
        req = Request(endpoint2)
        try:
            res = urlopen(req)
            outs = json.loads(res.read())
            for imp in values:
                try:
                    NDC_dict[imp] = outs['results'][0]['openfda'][imp]
                except KeyError:
                    NDC_dict[imp] = 'not_found'
        except HTTPError as e:
            pass
    return(NDC_dict)


def query_ndc(df, dict_name):
    time_counter=0
    
    for i in range(df.shape[0]):
        time_counter += 1
        #if not in the dictionary, put it in dictionary,
        if df.iloc[i].Product_Name not in dict_name.keys():
            dict_name[df.iloc[i].Product_Name] = get_info(df.iloc[i].Labeler_Code, df.iloc[i].Product_Code)
        
        #If it IS in dictionary, and the values are empty, put this one in 
        elif not bool(dict_name[df.iloc[i].Product_Name]):
            dict_name[df.iloc[i].Product_Name] = get_info(df.iloc[i].Labeler_Code, df.iloc[i].Product_Code)
        #If the data is already in there, just pass 
        else:
            pass
        
        if time_counter%240 == 0:
            time.sleep(60)
        
    return dict_name

# TROUBLESHOOTING

Need to reduce # of nulls: 3509

time.sleep to sleep every 240 sends for 10s: Nulls 2354

time.sleep to sleep every 240 sends for 15s : Nulls 1940

240 sends for 60s : Nulls 1189

________

some drugs do not show up in NDC directory even by searching online codes ie. 54162-0600 / HYDROCERIN
?????


In [434]:
deduped.shape

(21152, 13)

In [416]:
# QUERY NDC WORKS BECAUSE STRATTERA HAS 6 ENTRIES WITH UNIQUE L,P COMBOS, RETURNS DICTIOANRY WITH ONLY ONE
stratd = {}
query_ndc(deduped[deduped.Product_Name == 'STRATTERA'], stratd)

{'STRATTERA': {'brand_name': ['Strattera'],
  'generic_name': ['ATOMOXETINE HYDROCHLORIDE'],
  'manufacturer_name': ['Eli Lilly and Company'],
  'product_type': ['HUMAN PRESCRIPTION DRUG']}}

# Find Unique ['Product_Name'] for ['Labeler_Code', 'Product_Code']

In [463]:
deduped = unsup.drop_duplicates(subset=['Labeler_Code', 'Product_Code'])
dd_dict={}
query_ndc(deduped, dd_dict)
pd.to_pickle(dd_dict, 'drug_dictionary_60sleep.pkl')

In [474]:
pd.DataFrame(dd_dict).T.isnull().sum()

brand_name           1189
generic_name         1189
manufacturer_name    1189
product_type         1189
dtype: int64

In [470]:
outs = pd.DataFrame(dd_dict).T

In [471]:
outs

,brand_name,generic_name,manufacturer_name,product_type
STRATTERA,[Strattera],[ATOMOXETINE HYDROCHLORIDE],[Eli Lilly and Company],[HUMAN PRESCRIPTION DRUG]
CYMBALTA,[Cymbalta],[DULOXETINE HYDROCHLORIDE],[Eli Lilly and Company],[HUMAN PRESCRIPTION DRUG]
EVISTA,[Evista],[RALOXIFENE HYDROCHLORIDE],[Eli Lilly and Company],[HUMAN PRESCRIPTION DRUG]
EFFIENT,[Effient],[PRASUGREL HYDROCHLORIDE],[Eli Lilly and Company],[HUMAN PRESCRIPTION DRUG]
HUMALOG,[Humalog],[INSULIN LISPRO],[Eli Lilly and Company],[HUMAN PRESCRIPTION DRUG]
GLUCAGON E,[Glucagon],[GLUCAGON],[Eli Lilly and Company],[HUMAN PRESCRIPTION DRUG]
HUMULIN R,[Humulin R],[INSULIN HUMAN],[Eli Lilly and Company],[HUMAN OTC DRUG]
HUMULIN N,[Humulin N ],[INSULIN HUMAN],[Eli Lilly and Company],[HUMAN OTC DRUG]
FORTEO,[Forteo],[TERIPARATIDE],[Eli Lilly and Company],[HUMAN PRESCRIPTION DRUG]
HUMULIN 70,[Humulin 70/30 ],[INSULIN HUMAN],[Eli Lilly and Company],[HUMAN OTC DRUG]


In [473]:
deduped[deduped.Product_Name == 'Idelvion 2']

,Utilization_Type,State,Labeler_Code,Product_Code,Package_Size,Year,Quarter,Product_Name,Units_Reimbursed,Number_of_Prescriptions,Total_Amount_Reimbursed,Medicaid_Amount_Reimbursed,Non_Medicaid_Amount_Reimbursed
14221251,FFSU,IL,69911,0867,2,2017,1,Idelvion 2,23.0,23.0,310268.27,56348.74,253919.53


In [454]:
outs[outs.brand_name.isnull() == True]

,brand_name,generic_name,manufacturer_name,product_type
FARXIGA,NaN,NaN,NaN,NaN
ONGLYZA,NaN,NaN,NaN,NaN
AUVI-Q,NaN,NaN,NaN,NaN
PROVENTIL,NaN,NaN,NaN,NaN
SEVELAMER,NaN,NaN,NaN,NaN
VIMPAT,NaN,NaN,NaN,NaN
CITRUCEL,NaN,NaN,NaN,NaN
RHINOCORT,NaN,NaN,NaN,NaN
KLOR-CON,NaN,NaN,NaN,NaN
ORA-SWEET-,NaN,NaN,NaN,NaN


In [404]:
L = '49281'
P = '190'

In [461]:
name='HYDROCERIN'
why(name)


{}

In [408]:
def why(name):
    return get_info(L =deduped[deduped.Product_Name == name].Labeler_Code.values[0], 
                    P = deduped[deduped.Product_Name == name].Product_Code.values[0])

In [ ]:
for i in range(d)

In [30]:
ny = unsup[unsup.State == 'NY']
# ny = ny[ny.NDC.str.startswith('0000') == True]
#Regex looks for anything that starts with 4 zeros, and reformats to fit with NDC endpoint at OPENFDA
#Only ~ 1300 Product_Names fit this format, much less when you unique 
# ny['ndc2'] = ny.NDC.str.replace('0{4}(?P<first>\d{1})(?P<four>\d{4})\d*', r"000\1-\2")
# small_ny = ny[['Product_Name', 'ndc2']]
# small_ny.drop_duplicates(inplace=True)